# Incidence Matrixes

In [1]:
sample_bbc_news_sentences = [
    "China confirms Interpol chief detained",
    "Turkish officials believe the Washington Post writer was killed in the Saudi consulate in Istanbul.",
    "US wedding limousine crash kills 20",
    "Bulgarian journalist killed in park",
    "Kanye West deletes social media profiles",
    "Brazilians vote in polarised election",
    "Bull kills woman at French festival",
    "Indonesia to wrap up tsunami search",
    "Tina Turner reveals wedding night ordeal",
    "Victory for Trump in Supreme Court battle",
    "Clashes at German far-right rock concert",
    "The Walking Dead, actor dies aged 76",
    "Jogger in Netherlands finds lion cub",
    "Monkey takes the wheel of Indian bus"
]

In [2]:
#basic tokenization
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()
sample_bbc_news_sentences_tokenized = [tokenizer.tokenize(sentence) 
                            for sentence in sample_bbc_news_sentences]
temp=[]
for sentence in sample_bbc_news_sentences:
    temp.append(tokenizer.tokenize(sentence))

sample_bbc_news_sentences_tokenized[0]

['China', 'confirms', 'Interpol', 'chief', 'detained']

In [3]:
sample_bbc_news_sentences_tokenized_lower = [[_t.lower() 
                                              for _t in _s] 
                for _s in sample_bbc_news_sentences_tokenized]

sample_bbc_news_sentences_tokenized_lower[0]


['china', 'confirms', 'interpol', 'chief', 'detained']

In [4]:
#get all unique tokens
# Step 1: Flatten (sum)
# Step 2: Put in SET

# [[token1,token2,...],[...],[...],...]
unique_tokens = set(sum(sample_bbc_news_sentences_tokenized_lower, []))
# [token1, token2,...,token_K+1]
list(unique_tokens)[:5]
len(unique_tokens)

83

In [5]:
# create incidence matrix (term-document frequency)
import numpy as np
incidence_matrix = np.array([[sent.count(token)
                      for sent in sample_bbc_news_sentences_tokenized_lower] 
                for token in unique_tokens])
a =[]
b = []
for sent in sample_bbc_news_sentences_tokenized_lower:
    for token in unique_tokens:
        a.append(sent.count(token))
    b.append(a)
    a = []
print(incidence_matrix)
#incidence_matrix.shape

[[0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]


In [6]:
incidence_matrix.shape

(83, 14)

![Inverted Index](img/inverted-index.png)

# Dataset 
https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups 

# We will now construct the Inverted Index - only the dictionary part (term and #docs)

In [7]:
from nltk.tokenize import sent_tokenize
from collections import defaultdict, Counter
from string import punctuation
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
document = "US wedding limousine crash kills 20. Kanye West, deletes social media profiles!"
def preprocess_document(content):
    """
    Returns a list of tokens for a document's content. 
    Tokens should not contain punctuation and should be lower-cased.
    """
    sentences = sent_tokenize(content)
    
    tokens = []
    for _sent in sentences:
        sent_tokens = tokenizer.tokenize(_sent)
        sent_tokens = [_tok.lower() for _tok in sent_tokens if _tok not in punctuation]
        
        tokens += (sent_tokens)
        
    return tokens
    pass

         
def prepare_dataset(documents_dir):
    """
    Returns list of documents in the documents_dir, where each document is a list of its tokens. 
    
    """
    tokenized_documents = []
    for document in os.listdir(documents_dir):
        with open(os.path.join(documents_dir, document), errors='ignore') as outf:
            tokenized_documents.append(preprocess_document(outf.read()))
            
    print("Found documents: ", len(tokenized_documents))
    return tokenized_documents
    pass

In [9]:
preprocess_document(document)

['us',
 'wedding',
 'limousine',
 'crash',
 'kills',
 '20',
 'kanye',
 'west',
 'deletes',
 'social',
 'media',
 'profiles']

In [10]:
prepare_dataset('./data/mini_newsgroups/sci.electronics')

Found documents:  100


[['newsgroups',
  'sci.electronics',
  'path',
  'cantaloupe.srv.cs.cmu.edu',
  'crabapple.srv.cs.cmu.edu',
  'fs7.ece.cmu.edu',
  'europa.eng.gtefsd.com',
  'paladin.american.edu',
  'darwin.sura.net',
  'news-feed-1.peachnet.edu',
  'gatech',
  'swrinde',
  'sdd.hp.com',
  'decwrl',
  'decwrl',
  'uunet',
  'pipex',
  'doc.ic.ac.uk',
  'agate',
  'dog.ee.lbl.gov',
  'hellgate.utah.edu',
  'csn',
  'teal.csn.org',
  'et',
  'from',
  'et@teal.csn.org',
  'eric',
  'h',
  'taylor',
  'subject',
  're',
  'electronic',
  'tesla',
  'coils',
  'message-id',
  '<c4s0g8.dxv@csn.org>',
  'followup-to',
  'sci.electronics',
  'summary',
  'real',
  'world',
  'applications',
  'keywords',
  'tesla',
  'coil',
  'osc',
  'flyback',
  'transformers',
  'wireless',
  'emi',
  'ac',
  'ignition',
  'sender',
  'eric',
  'h',
  'taylor',
  'nntp-posting-host',
  'teal.csn.org',
  'organization',
  '4',
  'l',
  'laboratories',
  'references',
  '<1993mar23.040116.14355@norfolk.vak12ed.edu>',
  '<

In [11]:
def document_frequency(tokenized_documents):
    """
    Returns a dictionary {token:number of documents containing the token}
    """
    doc_freq = defaultdict(lambda: 0)
    for _document in tokenized_documents:
        for _token in set(_document):
            doc_freq[_token] += 1
    return doc_freq

In [12]:
selected_category = 'data/mini_newsgroups/sci.crypt/'
print(selected_category)
tokenized_dataset = prepare_dataset(selected_category)
print("Sample tokenized document:")
print(tokenized_dataset[0][:10])

df = document_frequency(tokenized_dataset)
print(df)

data/mini_newsgroups/sci.crypt/
Found documents:  100
Sample tokenized document:
['path', 'cantaloupe.srv.cs.cmu.edu', 'crabapple.srv.cs.cmu.edu', 'fs7.ece.cmu.edu', 'europa.eng.gtefsd.com', 'news.ans.net', 'cmcl', '2', 'arizona', 'ho']
defaultdict(<function document_frequency.<locals>.<lambda> at 0x000002001C9A7380>, {'can': 55, 'implementations': 1, 'path': 100, 'public': 29, '00:12': 1, 'licensing': 1, '<930403152101.890833@dockmaster.ncsc.mil>': 1, 'lines': 99, 'on': 58, 'fs7.ece.cmu.edu': 23, 'knife': 1, '2': 20, 'key': 40, 'what': 44, 'with': 58, 'that': 79, 'of': 92, 'never': 6, 'involvement': 1, 'u': 15, 'butcher': 1, 'forthright': 1, 'orman': 1, 'creation': 2, 'it': 77, 'regard': 5, 'europa.eng.gtefsd.com': 23, 'sci.crypt': 100, 'arizona': 1, 'news.ans.net': 1, 'apr': 100, 'the': 97, 'to': 92, 'cantaloupe.srv.cs.cmu.edu': 100, 'find': 14, 're': 81, 'cmcl': 1, 'message-id': 100, 'date': 100, 'ho@cs.arizona.edu': 1, '6': 4, 'attribute': 1, 'from': 100, 'news@cs.arizona.edu': 1, 

In [13]:

print("Most common words:")
print(Counter(df).most_common(10))
print("Least common words:")
print(Counter(df).most_common()[-10:])

Most common words:
[('path', 100), ('sci.crypt', 100), ('apr', 100), ('cantaloupe.srv.cs.cmu.edu', 100), ('message-id', 100), ('date', 100), ('from', 100), ('subject', 100), ('newsgroups', 100), ('lines', 99)]
Least common words:
[('nato', 1), ('driver', 1), ('led', 1), ('briefings', 1), ('magnetic', 1), ('dotmatrix', 1), ('shared', 1), ('codeword', 1), ('eighties', 1), ('emission', 1)]


# Extract Pairs of (token, document_id) tuples 
These will eventually end up sorted by document_id.

In [16]:
from os import scandir # can be used for easier iteration of documents in a folder
# can check is_file() on the objects returned by scan_dir 
# contain whole document path, so no need to join with the directory

import os

def get_token_doc_id_pairs(category_dir):
    """
    Iteratively goes through the documents in the category_dir and constructs/returns:
    1. A list of (token, doc_id) tuples
    2. A dictionary of doc_id:doc_name
    """
    tkn_docid = []
    docid_docnm = {}
    for i, document in enumerate(os.listdir(category_dir)):
        with open(os.path.join(category_dir, document), errors='ignore') as outf:
            tkn_docid += [(token,i) for token in preprocess_document(outf.read())]
        docid_docnm[i] = document
            
    return (tkn_docid, docid_docnm)

In [33]:
token_docid, doc_ids = get_token_doc_id_pairs('data/mini_newsgroups/rec.autos/')

In [34]:
print(doc_ids[2])

101592


In [35]:
token_docid[:10]

[('path', 0),
 ('cantaloupe.srv.cs.cmu.edu', 0),
 ('das-news.harvard.edu', 0),
 ('ogicse', 0),
 ('uwm.edu', 0),
 ('wupost', 0),
 ('uunet', 0),
 ('world', 0),
 ('edwards', 0),
 ('from', 0)]

__Example output:__ <br>
token_docid, doc_ids = get_token_doc_id_pairs('data/mini_newsgroups/rec.autos/')<br>
print(doc_ids[2])<br>
token_docid[:10]<br>

> DirEntry '101577' <br>
>[('newsgroups', 0),
> ('rec.autos', 0),
> ('path', 0),
> ('cantaloupe.srv.cs.cmu.edu', 0),
> ('magnesium.club.cc.cmu.edu', 0),
> ('news.sei.cmu.edu', 0),
> ('fs7.ece.cmu.edu', 0),
> ('europa.eng.gtefsd.com', 0),
> ('howland.reston.ans.net', 0),
> ('ux1.cso.uiuc.edu', 0)]

# Sort by token

In [36]:
from operator import itemgetter
from collections import defaultdict
sorted_token_docid = sorted(token_docid, key=itemgetter(0))
sorted_token_docid[-10:]

[('zaphod.mps.ohio-state.edu', 97),
 ('zaphod.mps.ohio-state.edu', 98),
 ('zaphod.mps.ohio-state.edu', 99),
 ('zauberer', 57),
 ('zeolite', 40),
 ('zip', 49),
 ('zip', 49),
 ('zx', 79),
 ('zx-r', 94),
 ("|'8", 71)]

# Merge token occurrences in a single document -> (token, doc_id, term_freq) tuples

In [51]:
def merge_token_in_doc(sorted_token_docid):
    """
    Returns a list of (token, doc_id, term_freq) tuples from a sorted list of (token, doc_id) list, 
    where if a token appears n times in a doc_id, we merge it in a tuple (toke, doc_id, n).
    """
    tkn_docid_tf = []
    temp_dict = defaultdict(lambda: 0)
    for tkn, doc_id in sorted_token_docid:
        temp_dict[tkn + '##' + str(doc_id)] += 1
    for merged_str in temp_dict:
        split = merged_str.split('##')
        tkn_docid_tf.append((split[0],split[1],temp_dict[merged_str]))
    
    return tkn_docid_tf

def merge_token_in_doc_v2(sorted_token_docid):
    merged_tokens_in_doc = []
    for token, doc_id in sorted_token_docid:
        if merged_tokens_in_doc:
            prev_tok, prev_doc_id, prev_freq = merged_tokens_in_doc[-1]
            if prev_tok == token and prev_doc_id == doc_id:     
                merged_tokens_in_doc[-1] = (token, doc_id, prev_freq+1)
            else:
                merged_tokens_in_doc.append((token, doc_id, 1))
        else:
            merged_tokens_in_doc.append((token, doc_id, 1))
    return merged_tokens_in_doc

In [52]:
merged_token_in_docs = merge_token_in_doc_v2(sorted_token_docid)
merged_token_in_docs[-10:]

[('zaphod.mps.ohio-state.edu', 96, 1),
 ('zaphod.mps.ohio-state.edu', 97, 1),
 ('zaphod.mps.ohio-state.edu', 98, 1),
 ('zaphod.mps.ohio-state.edu', 99, 1),
 ('zauberer', 57, 1),
 ('zeolite', 40, 1),
 ('zip', 49, 2),
 ('zx', 79, 1),
 ('zx-r', 94, 1),
 ("|'8", 71, 1)]

__Example output:__ <br>

merge_token_in_doc = merge_token_in_doc(sorted_token_docid) <br>
merged_tokens_in_doc[-10:] <br>

>[('zaphod.mps.ohio-state.edu', 96, 1),
 ('zaphod.mps.ohio-state.edu', 97, 1),
 ('zaphod.mps.ohio-state.edu', 98, 1),
 ('zaphod.mps.ohio-state.edu', 99, 1),
 ('zauberer', 47, 1),
 ('zeolite', 49, 1),
 ('zip', 77, 2),
 ('zx', 86, 1),
 ('zx-r', 57, 1),
 ("|'8", 70, 1)]

# Split into Dictionary and Postings (usually linked lists for each word)

In [53]:
from collections import defaultdict
dictionary = defaultdict(lambda: (0, 0)) # term : doc_freq, tot freq
postings = defaultdict(lambda: []) # term: doc_ids, doc_freq

for token, doc_id, doc_freq in merged_token_in_docs:
    dictionary[token] = (dictionary[token][0]+1, dictionary[token][1]+doc_freq)

# usually implemented as linked lists
for token, doc_id, doc_freq in merged_token_in_docs:
    postings[token].append((doc_id, doc_freq)) 

In [54]:
dictionary['zip'], dictionary['zaphod.mps.ohio-state.edu']

((1, 2), (51, 52))

In [55]:
postings['zip'], postings['zaphod.mps.ohio-state.edu'][:3]

([(49, 2)], [(3, 1), (14, 1), (17, 1)])

# Boolean Queries in the index

## AND query: we want to find documents which contain both 'living' and 'dead' in them.

We use a merging algorithm for conjunction queries, which simultaneously traverses the postings of the given words.

- Living: 1 -> 2 -> 5 -> 17 -> 30 -> 31 -> 44 -> 45 -> 47
- Dead: 5 -> 17 -> 44
- Intersection: 5 -> 17 -> 44

It takes __linear time__ over the number of documents the two words appear in. <br/>
It is important to have postings for each word __sorted by document id__.

In [ ]:
def and_query(postings, word1, word2):
    """
    merging postings lists of two words
    """
    postings_word1 = postings[word1]
    postings_word2 = postings[word2]
    
    documents_results = []
    
    postings_ind1, postings_ind2 = 0, 0
    while postings_ind1 < len(postings_word1) and postings_ind2 < len(postings_word2):
        doc_id1, doc_id2 = postings_word1[postings_ind1][0], postings_word2[postings_ind2][0]
        if doc_id1 == doc_id2:
            documents_results.append(postings_word1[postings_ind1][0])
            postings_ind1 += 1
            postings_ind2 += 1
        elif doc_id1 < doc_id2:
            postings_ind1 += 1
        elif doc_id1 > doc_id2:
            postings_ind2 += 1
    return documents_results

In [144]:
doc_id = and_query(postings, 'living', 'dead')
doc_id

[36]

In [145]:
postings['living']

[(30, 1), (36, 1), (78, 1)]

In [146]:
postings['dead']

[(29, 1), (36, 1), (65, 1)]

__Example Output:__ <br>

doc_id = and_query(postings, 'living', 'dead') <br>
doc_ids[doc_id[0]] <br>

> DirEntry '102983'

## Questions:

- How about if we want to find a document containing N words? 
- What will be the execution time for queries with NOT/ OR and different combinations?
- What are the downsides of boolean queries?

## Advanced query features
* We can optimize with : processing in order of increasing freq; skip pointers
* Proximity
* Zones
* Phrase queries (bi-word indexes, positional indexes)